In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28916
2,Huelva,Confirmados PDIA 14 días,1009
3,Huelva,Tasa PDIA 14 días,"196,62100278660094"
4,Huelva,Confirmados PDIA 7 días,463
5,Huelva,Total Confirmados,29092
6,Huelva,Curados,25440
7,Huelva,Fallecidos,369


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28916.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7525.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1020 personas en los últimos 7 días 

Un positivo PCR cada 519 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28916.0,463.0,1009.0,513170.0,90.223513,196.621003,76.0
Condado-Campiña,9173.0,220.0,489.0,156231.0,140.817123,312.998061,43.0
Huelva-Costa,16830.0,212.0,439.0,289548.0,73.217567,151.615622,30.0
Huelva (capital),7525.0,141.0,277.0,143837.0,98.027628,192.579100,28.0
Almonte,1579.0,82.0,155.0,24507.0,334.598278,632.472355,15.0
Bollullos Par del Condado,856.0,48.0,83.0,14387.0,333.634531,576.909710,8.0
Rociana del Condado,525.0,15.0,29.0,7939.0,188.940673,365.285300,7.0
Palma del Condado (La),1044.0,8.0,15.0,10801.0,74.067216,138.876030,5.0
San Juan del Puerto,454.0,7.0,22.0,9411.0,74.381043,233.768994,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,233.0,2.0,33.0,3261.0,61.330880,1011.959522,0.0
Cumbres de San Bartolomé,11.0,1.0,3.0,376.0,265.957447,797.872340,0.0
Almonte,1579.0,82.0,155.0,24507.0,334.598278,632.472355,15.0
Bollullos Par del Condado,856.0,48.0,83.0,14387.0,333.634531,576.909710,8.0
Galaroza,30.0,3.0,6.0,1382.0,217.076700,434.153401,0.0
Beas,201.0,3.0,17.0,4341.0,69.108500,391.614835,0.0
Rociana del Condado,525.0,15.0,29.0,7939.0,188.940673,365.285300,7.0
Condado-Campiña,9173.0,220.0,489.0,156231.0,140.817123,312.998061,43.0
Bonares,257.0,4.0,17.0,6060.0,66.006601,280.528053,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,233.0,2.0,33.0,3261.0,61.330880,1011.959522,0.0,0.060606
Trigueros,268.0,1.0,10.0,7862.0,12.719410,127.194098,0.0,0.100000
Cortegana,200.0,1.0,6.0,4602.0,21.729683,130.378096,0.0,0.166667
Beas,201.0,3.0,17.0,4341.0,69.108500,391.614835,0.0,0.176471
Bonares,257.0,4.0,17.0,6060.0,66.006601,280.528053,1.0,0.235294
Rosal de la Frontera,29.0,1.0,4.0,1697.0,58.927519,235.710077,0.0,0.250000
San Juan del Puerto,454.0,7.0,22.0,9411.0,74.381043,233.768994,3.0,0.318182
Almonaster la Real,66.0,1.0,3.0,1788.0,55.928412,167.785235,0.0,0.333333
Minas de Riotinto,114.0,1.0,3.0,3812.0,26.232949,78.698846,0.0,0.333333
